In [1]:
# Yernar Shambayev, DL-2

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torchvision.models import resnet18, vgg16
import os, time

In [2]:
# 3.Добавьте аугментацию данных
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names_val = image_datasets['val'].classes
print(class_names, class_names_val)
print(len(image_datasets['train']), len(image_datasets['val']))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

{'train': 244, 'val': 153}
['ants', 'bees'] ['ants', 'bees']
244 153


In [4]:
# Общая функция для моделей
def training(name, model, criterion, optimizer):
    start = time.time()
    num_epochs = 3
    print(name)

    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Эпоха: {epoch+1} / {num_epochs}')

        for mode in ['train', 'val']:
            if mode == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[mode]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(mode == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if mode == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[mode]
            epoch_acc = running_corrects.double() / dataset_sizes[mode]

            print(f'Режим: {mode}, потери: {epoch_loss:.4f}, точность: {epoch_acc:.4f}')

    time_elapsed = time.time() - start
    print(f'Прошло: {time_elapsed // 60:.0f} мин {time_elapsed % 60:.0f} сек')
    print('')

In [5]:
# 1. Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)
model = models.resnet18(pretrained=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
training('resnet18 без FineTuning', model, criterion, optimizer)

model = models.vgg16(pretrained=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
training('vgg16 без FineTuning', model, criterion, optimizer)

# 2. Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)
model = models.resnet18(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
training('resnet18 с FineTuning', model, criterion, optimizer)

model = models.vgg16(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
training('vgg16 с FineTuning', model, criterion, optimizer)

resnet18 без FineTuning
Эпоха: 1 / 3
Режим: train, потери: 1.7193, точность: 0.4959
Режим: val, потери: 0.9619, точность: 0.5752
Эпоха: 2 / 3
Режим: train, потери: 0.7339, точность: 0.6393
Режим: val, потери: 0.7154, точность: 0.5556
Эпоха: 3 / 3
Режим: train, потери: 0.7214, точность: 0.6025
Режим: val, потери: 0.6720, точность: 0.6601
Прошло: 0 мин 12 сек

vgg16 без FineTuning
Эпоха: 1 / 3
Режим: train, потери: 3.5965, точность: 0.4221
Режим: val, потери: 1.1276, точность: 0.4575
Эпоха: 2 / 3
Режим: train, потери: 1.2453, точность: 0.5041
Режим: val, потери: 0.8698, точность: 0.5425
Эпоха: 3 / 3
Режим: train, потери: 0.8379, точность: 0.5164
Режим: val, потери: 0.8271, точность: 0.4575
Прошло: 0 мин 45 сек

resnet18 с FineTuning
Эпоха: 1 / 3
Режим: train, потери: 1.7307, точность: 0.6393
Режим: val, потери: 0.4804, точность: 0.8954
Эпоха: 2 / 3
Режим: train, потери: 0.9449, точность: 0.7664
Режим: val, потери: 1.0811, точность: 0.6732
Эпоха: 3 / 3
Режим: train, потери: 0.7251, точнос

In [ ]:
# Вывод: FineTuning улучшил точность 